# Web Scraper for CMS Medicare Preventive Services

### Description

#### Page that we're scraping: 
https://www.cms.gov/Medicare/Prevention/PrevntionGenInfo/medicare-preventive-services/MPS-QuickReferenceChart-1.html

#### Purpose
Scrape the website for CMS Medicare preventive services (30 pages, one for each service, as of Jan 2019). This will consolidate the HCPCS codes, diagnosis codes, and waived vs. applied member payments into a spreadsheet. These codes will help carve out preventive services for claims grouping.

#### Output
* Excel tab with a grid of all attributes (e.g., covered hcpcs and icd codes) for each service as printed on the site
* Excel tab with hcpcs codes formatted into a comma-separated list that's friendlier for the grouper
* Run time: 6-8 minutes for 30 links; <1 minute if testing on 1-2 links.

#### Background 
* Our website has JavaScript that produces HTML. 
    * This means we need a browser to render the JavaScript for the final HTML. 
    * Viewing the source HTML code will not get us the final rendered HTML that a user interacts with on their browser.
* There is one home URL and each individual service has a URL that is the home URL concatenated with the service alias. E.g., "home_url.com.html#service"

#### Steps
* Use BeautifulSoup to scrape the CMS site for names and aliases of services (Alcohol Misuse = 'alc_misuse'). Store these in a dictionary called 'services'.
* Create a dictionary call 'svc_dict'with each service as a key and store it's link.
    * We will later open via Selenium/Firefox. Do this by appending the service to the base/home URL.
    * We will also add other attributes to this dictionary like HCPCS and ICD codes that are relevent to the service
* Create a function get_info(service, link) that opens a URL, creates a selenium browser object, and fetches the elements we want and stores those in our 'svc_dict' for one given service.
* iterate through all links and services and call get_info() to populate our dictionary
* lastly, create 'svc_dict2' which will do some text parsing on hcpcs codes only to make a neater comma-separated list of hcpcs codes related to each service
* convert 'svc_dict' and 'svc_dict2' to dataframes and export to excel (two tabs)


#### Pre-requisites
* install selenium and beautifulsoup4 packages
    * e.g., "conda install selenium"
    * "conda install beatifiulsoup4"
* beautifulsoup supports the Firefox browser, but you need to download a geckodriver.exe and store the executable where your python path is located
    * Download here: https://github.com/mozilla/geckodriver/releases/tag/v0.21.0
    * Try v21 geckodriver first. The latest is v23, but it seems to cause issues on certain Windows PCs

In [1]:
# geckodriver v21.0 appears to work while v23.0 does not...
# rationale for selenium vs. beautifulsoup: the web page we want to scrape uses javascript. 
    # we need selenium to launch the browser and render the final html 
    # we use beautifulsoup on source html that does not require the browser to render it

In [2]:
import requests
import bs4
import os
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
import pandas as pd
import pprint

In [3]:
#Store user login (e.g., "jsmith") in variable called "user".
#We'll use this to guide the Excel output to the user's Downloads folder.

user = os.getlogin()

In [4]:
#create some variables that will be used as arguments when we call webdriver.Firefox() later
    # geckodriver path is needed for selenium to work with firefox
    # binary is used to point selenium to the Firefox executable
    #later, we create a browser object using gecko_path and binary as arguments
    #e.g., "browser = webdriver.Firefox(firefox_binary=binary, executable_path=gecko_path)"
    #the standard implementation can be simplified to "webdriver.Firefox()" depending on your PC

gecko_path = r'C:\Users\{}\AppData\Local\Continuum\anaconda3\geckodriver.exe'.format(user)
binary = FirefoxBinary(r'C:\Users\{}\AppData\Local\Mozilla Firefox\firefox.exe'.format(user))

In [5]:
#use BeautifulSoup to scrape source html for list of services

res = requests.get('https://www.cms.gov/Medicare/Prevention/PrevntionGenInfo/medicare-preventive-services/MPS-QuickReferenceChart-1.html')
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text,'lxml')

In [6]:
# this will fetch a list of html objects that we can parse for each service

services_html = soup.select("li[role='presentation']")

In [7]:
# we want the text after id= ("alc_misuse" in this case) for each element

services_html[1]

<li role="presentation"><button class="telehealthButton" id="ALC_MISUSE" role="menuitem" tabindex="0" title="View Codes and Information About Alcohol Misuse Screening and Counseling" type="button">Alcohol Misuse Screening and <span class="noWrap">Counseling <img alt="This is a Telehealth Service" src="images/Telehealth-Service.png"/></span></button></li>

In [8]:
# create list of services (aliases) that will be appended to a base url link for iteration
# start at one b/c the zero-th element is "all_services" which is not a service
services = []

for i in range(1,len(services_html)):
    z_str = str([x for x in str(services_html[i]).split(' ') if 'id' in x])
    service = z_str.split('"')[1]
    services.append(service)
    
services[0]

'ALC_MISUSE'

In [9]:
# of service
len(services)

30

In [10]:
# parse for full descriptions. E.g., "alc_misuse" = "Alcohol Misuse Screening and Counseling"

all_services = soup.select("#ALL_SERVICES")

In [11]:
# we want the text "Alcohol Misuse Screening and Counseling"
# index [1] limites to on service, but we will iterate and get all the service in For Loop below

str(all_services).split('<input id=')[1].split('for=')

['"ALC_MISUSE_PRINT" name="printOption" tabindex="0" type="checkbox" value="AlcMisuse"/><label ',
 '"ALC_MISUSE_PRINT">Alcohol Misuse Screening and Counseling</label>\n']

In [12]:
for i in range(1,len(services)):
    s = str(all_services).split('<input id=')[i].split('for=')
    s = [s.split('">') for s in s]
    print(s[1][0][1:-6], s[1][1][:-9], sep=', ')

ALC_MISUSE, Alcohol Misuse Screening and Counseling
AWV, Annual Wellness Visit
BONE_MASS, Bone Mass Measurements
CARDIO_DIS, Cardiovascular Disease Screening Tests
COLO_CAN, Colorectal Cancer Screening
TOBACCO, Counseling to Prevent Tobacco Use
DEPRESSION, Depression Screening
DIABETES, Diabetes Screening
DIABETES_SELF, Diabetes Self-Management Training
GLAUCOMA, Glaucoma Screening
HEP_B_SCREEN, HBV Screening
HEP_B, HBV Vaccine and Administration
HEP_C, HCV Screening
HIV, HIV Screening
FLU, Influenza Virus Vaccine and Administration
IPPE, IPPE
CARDIO_IBT, IBT for Cardiovascular Disease
OBESITY_IBT, IBT for Obesity
LUNG_CAN, Lung Cancer Screening
MNT, Medical Nutrition Therapy
PROLONGED, Prolonged Preventive Services
MDPP, Medicare Diabetes Prevention Program Expanded Model
PNEUMO, Pneumococcal Vaccine and Administration
PROSTATE_CAN, Prostate Cancer Screening
CERV_CAN, Screening for Cervical Cancer with HPV Tests
STI, Screening for STIs and HIBC to Prevent STIs
MAMMO, Screening Mammogr

In [13]:
# create nested dict for each service that includes URL link for now
# populate with other service-level attributes later like: hcpcs, icd10, etc

base_link = r'https://www.cms.gov/Medicare/Prevention/PrevntionGenInfo/medicare-preventive-services/MPS-QuickReferenceChart-1.html'

svc_dict = {}

for s in services:
    link = base_link+'#'+s
    svc_dict[s] = {}
    svc_dict[s]['link'] = link

In [14]:
# show all service keys

svc_dict.keys()

dict_keys(['ALC_MISUSE', 'AWV', 'BONE_MASS', 'CARDIO_DIS', 'COLO_CAN', 'TOBACCO', 'DEPRESSION', 'DIABETES', 'DIABETES_SELF', 'GLAUCOMA', 'HEP_B_SCREEN', 'HEP_B', 'HEP_C', 'HIV', 'FLU', 'IPPE', 'CARDIO_IBT', 'OBESITY_IBT', 'LUNG_CAN', 'MNT', 'MDPP', 'PNEUMO', 'PROLONGED', 'PROSTATE_CAN', 'CERV_CAN', 'STI', 'MAMMO', 'PAP', 'PELVIC', 'ULTRASOUND'])

In [15]:
# sample attributes within one service
# right now we only have link, but we will add other key-values for other attributes

svc_dict['ALC_MISUSE']

{'link': 'https://www.cms.gov/Medicare/Prevention/PrevntionGenInfo/medicare-preventive-services/MPS-QuickReferenceChart-1.html#ALC_MISUSE'}

In [16]:
# create function that iterates through attributes (hcpcs, icd10, etc) for a given web page
# add each attribute to the service dictionary

# take a link, open Firefox page
# find each element (HCPCS, ICD10, etc) within the rendered html text for the link 

def get_info(service, link):
    #browser = webdriver.Firefox() # may need more precise args within ()
    browser = webdriver.Firefox(firefox_binary=binary, executable_path=gecko_path)
    browser.get(link)
    elems = ['HCPCS', 'ICD10', 'WHO', 'FREQUENCY', 'PAYS']
    time.sleep(3)
    for e in elems:
        svc_dict[service][e] = browser.find_element_by_id(e).text
    browser.close()

In [17]:
# test get_info() on one service; iterate later

get_info('ALC_MISUSE', svc_dict['ALC_MISUSE']['link'])
svc_dict['ALC_MISUSE']

{'FREQUENCY': 'Frequency\nAnnually for G0442 (screening)\nFor those who screen positive, 4 times per year for G0443 (counseling)',
 'HCPCS': 'What’s Changed?\nNo 2019 second quarter changes\nHCPCS/CPT Codes\nG0442 – \nAnnual alcohol misuse screening, 15 minutes\nG0443 – \nBrief face-to-face behavioral counseling for alcohol misuse, 15 minutes',
 'ICD10': 'ICD-10 Codes\nNOTE:\nAdditional ICD-10 codes may apply. See the CMS ICD-10 webpage for individual Change Requests (CRs) and the specific ICD-10-CM codes Medicare covers for this service, and contact your Medicare Administrative Contractor (MAC) for guidance.',
 'PAYS': 'Medicare Beneficiary Pays\nCopayment/coinsurance waived\nDeductible waived',
 'WHO': 'Who Is Covered\nAll Medicare beneficiaries are eligible for alcohol screening.\nMedicare beneficiaries are eligible for counseling if they:\nScreen positive (those who misuse alcohol but whose levels or patterns of alcohol consumption do not meet criteria for alcohol dependence)\nAre 

In [18]:
# limit to two services for testing (to avoid iterating over 30)
# comment this out to include all services

# for k in list(svc_dict.keys()):
#     if k not in ['ALC_MISUSE', 'AWV', 'BONE_MASS']:
#         del svc_dict[k]
        
# svc_dict.keys()

In [19]:
%%time

# iterate through each service and url link
# populate service dictionary by calling get_info() each time
# print status

count = 0

for k in svc_dict.keys():
    count += 1
    print("{}  of {}: {}".format(count, len(svc_dict), k))
    try:
        get_info(k,svc_dict[k]['link'])
    except:
        print("Error for {}.".format(k))

1  of 30: ALC_MISUSE
2  of 30: AWV
3  of 30: BONE_MASS
4  of 30: CARDIO_DIS
5  of 30: COLO_CAN
6  of 30: TOBACCO
7  of 30: DEPRESSION
8  of 30: DIABETES
9  of 30: DIABETES_SELF
10  of 30: GLAUCOMA
11  of 30: HEP_B_SCREEN
12  of 30: HEP_B
13  of 30: HEP_C
14  of 30: HIV
15  of 30: FLU
16  of 30: IPPE
17  of 30: CARDIO_IBT
18  of 30: OBESITY_IBT
19  of 30: LUNG_CAN
20  of 30: MNT
21  of 30: MDPP
22  of 30: PNEUMO
23  of 30: PROLONGED
24  of 30: PROSTATE_CAN
25  of 30: CERV_CAN
26  of 30: STI
27  of 30: MAMMO
28  of 30: PAP
29  of 30: PELVIC
30  of 30: ULTRASOUND
Wall time: 5min 45s


In [20]:
pprint.pprint(svc_dict['ALC_MISUSE'])

{'FREQUENCY': 'Frequency\n'
              'Annually for G0442 (screening)\n'
              'For those who screen positive, 4 times per year for G0443 '
              '(counseling)',
 'HCPCS': 'What’s Changed?\n'
          'No 2019 second quarter changes\n'
          'HCPCS/CPT Codes\n'
          'G0442 – \n'
          'Annual alcohol misuse screening, 15 minutes\n'
          'G0443 – \n'
          'Brief face-to-face behavioral counseling for alcohol misuse, 15 '
          'minutes',
 'ICD10': 'ICD-10 Codes\n'
          'NOTE:\n'
          'Additional ICD-10 codes may apply. See the CMS ICD-10 webpage for '
          'individual Change Requests (CRs) and the specific ICD-10-CM codes '
          'Medicare covers for this service, and contact your Medicare '
          'Administrative Contractor (MAC) for guidance.',
 'PAYS': 'Medicare Beneficiary Pays\n'
         'Copayment/coinsurance waived\n'
         'Deductible waived',
 'WHO': 'Who Is Covered\n'
        'All Medicare beneficiaries 

In [21]:
print(svc_dict['ALC_MISUSE']['HCPCS'])

What’s Changed?
No 2019 second quarter changes
HCPCS/CPT Codes
G0442 – 
Annual alcohol misuse screening, 15 minutes
G0443 – 
Brief face-to-face behavioral counseling for alcohol misuse, 15 minutes


In [22]:
svc_df = pd.DataFrame.from_dict(svc_dict, orient='index')

In [23]:
svc_df.head()

,link,HCPCS,ICD10,WHO,FREQUENCY,PAYS
ALC_MISUSE,https://www.cms.gov/Medicare/Prevention/Prevnt...,What’s Changed?\nNo 2019 second quarter change...,ICD-10 Codes\nNOTE:\nAdditional ICD-10 codes m...,Who Is Covered\nAll Medicare beneficiaries are...,Frequency\nAnnually for G0442 (screening)\nFor...,Medicare Beneficiary Pays\nCopayment/coinsuran...
AWV,https://www.cms.gov/Medicare/Prevention/Prevnt...,What’s Changed?\nNo 2019 second quarter change...,ICD-10 Codes\nNOTE:\nAdditional ICD-10 codes m...,Who Is Covered\nAll Medicare beneficiaries who...,Frequency\nOnce in a lifetime for G0438 (first...,Medicare Beneficiary Pays\nG0438 and G0439:\nC...
BONE_MASS,https://www.cms.gov/Medicare/Prevention/Prevnt...,What’s Changed?\nNo 2019 second quarter change...,"ICD-10 Codes\nE21.0, E21.3, E23.0, E34.2, E89....",Who Is Covered\nCertain Medicare beneficiaries...,Frequency\nEvery 2 years\nMore frequently if m...,Medicare Beneficiary Pays\nCopayment/coinsuran...
CARDIO_DIS,https://www.cms.gov/Medicare/Prevention/Prevnt...,What’s Changed?\nNo 2019 second quarter change...,ICD-10 Codes\nZ13.6\nNOTE:\nAdditional ICD-10 ...,Who Is Covered\nAll Medicare beneficiaries wit...,Frequency\nOnce every 5 years,Medicare Beneficiary Pays\nCopayment/coinsuran...
CARDIO_IBT,https://www.cms.gov/Medicare/Prevention/Prevnt...,What’s Changed?\nNo 2019 second quarter change...,ICD-10 Codes\nNOTE:\nAdditional ICD-10 codes m...,Who Is Covered\nAll Medicare beneficiaries who...,Frequency\nAnnually,Medicare Beneficiary Pays\nCopayment/coinsuran...


### Parse hcpcs codes and place them in a dict

In [24]:
# setting to display all columns in pandas dataframe

pd.set_option('display.max_colwidth', -1)

In [25]:
# this dict will store each service as a key with the hcpcs codes as a string

svc_dict2 = {}

In [26]:
# display hcpcs column

svc_df['HCPCS'].head()

ALC_MISUSE    What’s Changed?\nNo 2019 second quarter changes\nHCPCS/CPT Codes\nG0442 – \nAnnual alcohol misuse screening, 15 minutes\nG0443 – \nBrief face-to-face behavioral counseling for alcohol misuse, 15 minutes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [27]:
# parse hcpcs from table for a given service

def get_hcpcs(service):
    s = svc_df['HCPCS'][svc_df.index == service].to_string(index=False)
    procs = [x for x in s.split('\\n') if '–' in x]
    procs = [x.replace(' – ','') for x in procs]
    #print(service,': {}'.format(','.join(procs)))
    svc_dict2[service] = {}
    svc_dict2[service]['HCPCS'] = ','.join(procs)
    return procs

In [28]:
for i in services:
    get_hcpcs(i)

In [29]:
svc_dict2['ALC_MISUSE']

{'HCPCS': 'G0442,G0443'}

In [30]:
for s in services:
    print('='*40)
    print(s)
    pprint.pprint(svc_dict2[s])

ALC_MISUSE
{'HCPCS': 'G0442,G0443'}
AWV
{'HCPCS': 'G0438,G0439,G0468,99497,99498'}
BONE_MASS
{'HCPCS': '76977,77078,77080,77081,77085,G0130'}
CARDIO_DIS
{'HCPCS': '80061,82465 Cholesterol, serum, total,83718 Lipoprotein, direct '
          'measurement, high density cholesterol (HDL cholesterol),84478 '
          'Triglycerides'}
COLO_CAN
{'HCPCS': '81528,82270,G0104,G0105,G0106,G0120,G0121,G0328,00812'}
TOBACCO
{'HCPCS': '99406,99407'}
DEPRESSION
{'HCPCS': 'G0444'}
DIABETES
{'HCPCS': '82947,82950,82951'}
DIABETES_SELF
{'HCPCS': 'G0108,G0109'}
GLAUCOMA
{'HCPCS': 'G0117,G0118'}
HEP_B_SCREEN
{'HCPCS': 'G0499,86704,86706,87340,87341'}
HEP_B
{'HCPCS': '90739,90740,90743,90744,90746,90747,G0010'}
HEP_C
{'HCPCS': 'G0472'}
HIV
{'HCPCS': '80081,G0432,G0433,G0435,G0475'}
FLU
{'HCPCS': '90630,90653,90654,90655,90656,90657,90658,90660,90661,90662,90672,90673,90674,90682,90685,90686,90687,90688,90689*,(*Effective '
          'for dates of service on or after January 1, 2019. See the Quarterly '
  

In [31]:
svc_df2 = pd.DataFrame.from_dict(svc_dict2, orient='index')
svc_df2.head()

,HCPCS
ALC_MISUSE,"G0442,G0443"
AWV,"G0438,G0439,G0468,99497,99498"
BONE_MASS,"76977,77078,77080,77081,77085,G0130"
CARDIO_DIS,"80061,82465 Cholesterol, serum, total,83718 Lipoprotein, direct measurement, high density cholesterol (HDL cholesterol),84478 Triglycerides"
CARDIO_IBT,G0446


In [32]:
writer = pd.ExcelWriter(r'C:\Users\{}\Downloads\preventive_services.xlsx'.format(user))
svc_df.to_excel(writer, 'Sheet1')
svc_df2.to_excel(writer, 'Sheet2')
writer.save()